In [1]:
#Colab Setup
%tensorflow_version 2.x

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!pip install pymorphy2

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 5.5MB/s 
Sat Jun 27 03:21:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| 

In [2]:
import sys
sys.path.append("/content/gdrive/My Drive/smu-ABAE/classes")

abae_training_data = "/content/gdrive/My Drive/smu-ABAE/data/abae_train.pkl"
emb_filename = '/content/gdrive/My Drive/smu-ABAE/models/w2v_embedding'
aspect_output_path = '/content/gdrive/My Drive/smu-ABAE/results/ABAE'

In [ ]:
# import sys
# import os

# module_path = os.path.abspath(os.path.join(".."))
# if module_path not in sys.path:
#     sys.path.append(module_path+"/scripts")
#     sys.path.append(module_path+"/classes")

# review_corpus_path = "../data/tech_review_sent_corpus.pkl"
# vocab_path = "../data/glove-tech-revew-vocab.txt"
# emb_filename = '../models/w2v_embedding'
# aspect_output_path = "../results/ABAE"
# vocab_path = "../data/vocab-text-review.txt"


## Parmeters

## Attention-Based-Aspect-Detection Code

In [5]:
from numpy.random import seed
import tensorflow as tf
seed(76244)
tf.random.set_seed(76244)

import ABAE.utils as U
parser = U.add_common_args()

Using TensorFlow backend.


In [15]:
import pickle
from keras.preprocessing import sequence

with open(abae_training_data,"rb") as f:
  vocab, train_x, overall_maxlen = pickle.load(f)

train_x = sequence.pad_sequences(train_x, maxlen=overall_maxlen)

vocab_inv = {}
for w, ind in vocab.items():
    vocab_inv[ind] = w

In [7]:
import os
from ABAE.w2vEmbReader import W2VEmbReader as EmbReader

emb_reader = EmbReader(emb_filename)

2020-06-27 03:21:57,570 INFO Loading embeddings from: /content/gdrive/My Drive/smu-ABAE/models/w2v_embedding
2020-06-27 03:21:57,571 INFO loading Word2VecKeyedVectors object from /content/gdrive/My Drive/smu-ABAE/models/w2v_embedding
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-06-27 03:21:59,001 INFO loading wv recursively from /content/gdrive/My Drive/smu-ABAE/models/w2v_embedding.wv.* with mmap=None
2020-06-27 03:21:59,002 INFO setting ignored attribute vectors_norm to None
2020-06-27 03:21:59,003 INFO loading vocabulary recursively from /content/gdrive/My Drive/smu-ABAE/models/w2v_embedding.vocabulary.* with mmap=None
2020-06-27 03:21:59,004 INFO loading trainables recursiv

  #vectors: 47833, #dimensions: 200


In [ ]:
import importlib
# import ABAE.model
# import ABAE.w2vEmbReader
import ABAE.train
# import ABAE.reader
# importlib.reload(ABAE.model)
# importlib.reload(ABAE.w2vEmbReader)
importlib.reload(ABAE.train)
# importlib.reload(ABAE.reader)

<module 'ABAE.train' from '/content/gdrive/My Drive/smu-ABAE/classes/ABAE/train.py'>

In [ ]:
import ABAE.model as m
from ABAE.train import train
from ABAE.optimizers import get_optimizer, sentence_batch_generator, negative_batch_generator
import itertools
from tqdm import tqdm
import json

emb_dim = 200
neg_sizes = [10,20,30,40,50]
aspect_sizes = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
ortho_regs = [0.01, 0.1, 0.25, 0.5, 0.75, 1]
batch_size = 50
epochs = 15
parameters = list(itertools.product(aspect_sizes, ortho_regs,neg_sizes))

optimizer = get_optimizer("adam")

for param in tqdm(parameters):
# for param in parameters:
    k = param[0]
    ortho_reg = param[1]
    neg_size = param[2]
    
    aspect_file_name = "abae-k-{0}-orth-{1}-neg-{2}".format(k, ortho_reg,neg_size)
    aspect_file_name = os.path.join(aspect_output_path,aspect_file_name)

    if not os.path.isfile(aspect_file_name):
      print("\n")
      print("="*80)
      print("Running for k={0}, ortho_reg={1}, neg_size={2}".format(k, ortho_reg, neg_size))
      model = m.create_model(overall_maxlen, vocab, k, neg_size, emb_reader, ortho_reg)
      # freeze the word embedding layer
      model.get_layer('word_emb').trainable = False
      model.compile(optimizer=optimizer, loss=U.max_margin_loss, metrics=[U.max_margin_loss])

      sen_gen = sentence_batch_generator(train_x, batch_size)
      neg_gen = negative_batch_generator(train_x, batch_size, neg_size)
      batches_per_epoch = len(train_x) // batch_size

      aspect, model = train(model, sen_gen, neg_gen, epochs, batch_size, batches_per_epoch, aspect_file_name, vocab_inv)

      model.save(aspect_file_name)
      with open(aspect_file_name + ".json", "w") as f:
          json.dump(aspect, f)
 
      print("="*80)
      break


  0%|          | 0/330 [00:00<?, ?it/s]



Running for k=5, ortho_reg=0.01, neg_size=10


In [14]:
print( aspect["Aspect 6"] )

['manager', 'smh', 'employee', 'nobody', 'slacker', 'hr', 'upper', 'management', 'apparent', 'consequence', 'rarely', 'lazy', 'raise', 'excuse', 'ceo', 'retaliate', 'actually', 'vp', 'often', 'tl', 'district', 'bonus', "n't", 'director', 'someone', 'paycheck', 'scar', 'fired', 'simply', 'punish', 'promised', 'dismissal', 'horribly', 'truth', 'guy', 'insult', 'unfairly', 'nonsense', 'payout', 'even', 'sadly', 'u', 'slack', 'dirt', 'anything', 'reprimand', 'incompetence', 'loser', 'threat', 'arbitrary', 'insulting', 'staff', 'seem', 'everyone', 'terminate', 'subordinate', 'victim', 'ridiculously', 'suddenly', 'minority', 'literally', 'blatant', 'let', 'strike', 'announce', 'supposedly', 'either', 'fault', 'somebody', 'hint', 'demote', 'warning', 'stats', 'unemployment', 'obviously', 'hero', 'pay', 'absence', 'garbage', 'disrespect', 'barely', 'dime', 'performer', 'mistake', 'ignore', 'merit', 'sick', 'forced', 'automatically', 'quota', 'recruiter', 'liar', 'reason', 'moral', 'pretend', '